Shift Contest "Займы"

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
import sklearn.metrics as metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from pandas.api.types import is_numeric_dtype
from catboost import CatBoostClassifier, Pool
import catboost as cb

## Change file path

In [2]:
df = pd.read_csv('C:/Users/ivans/Shift/lending/X_train.csv', index_col="index")
df1 = pd.read_csv('C:/Users/ivans/Shift/lending/X_test.csv', index_col="index")
y = pd.read_csv('C:/Users/ivans/Shift/lending/y_train.csv', index_col="index")
y_train=y['loan_status'].to_numpy()

In [3]:
df['fico_mean']=round((df['fico_range_high']+df['fico_range_low'])/2,0)
df['int_rate']=df['int_rate'].str.replace('%', '').astype(float)
df['revol_util']=df['revol_util'].str.replace('%', '').astype(float)
df['year_issue'] = pd.to_datetime(df['issue_d']).dt.year
df['year_earl'] = pd.to_datetime(df['earliest_cr_line']).dt.year

In [4]:
df1['fico_mean']=round((df1['fico_range_high']+df1['fico_range_low'])/2,0)
df1['int_rate']=df1['int_rate'].str.replace('%', '').astype(float)
df1['revol_util']=df1['revol_util'].str.replace('%', '').astype(float)
df1['year_issue'] = pd.to_datetime(df1['issue_d']).dt.year
df1['year_earl'] = pd.to_datetime(df1['earliest_cr_line']).dt.year

In [5]:
df.drop(['fico_range_high','fico_range_low','open_acc'], axis=1, inplace=True)

In [6]:
df1.drop(['fico_range_high','fico_range_low','open_acc'], axis=1, inplace=True)

In [7]:
pd.set_option('display.float_format', lambda x: '%0.4f' % x)
pd.set_option('display.max_columns', None)

In [8]:
for col in df.columns:
    print(f'{col} : {df[col].isna().sum()/df.shape[0]*100:.4f}%')

acc_now_delinq : 0.0000%
addr_state : 0.0000%
annual_inc : 0.0000%
chargeoff_within_12_mths : 0.0031%
collections_12_mths_ex_med : 0.0031%
delinq_2yrs : 0.0000%
dti : 0.0000%
earliest_cr_line : 0.0000%
emp_length : 5.9796%
funded_amnt : 0.0000%
home_ownership : 0.0000%
inq_last_12m : 47.2734%
installment : 0.0000%
int_rate : 0.0000%
issue_d : 0.0000%
loan_amnt : 0.0000%
mort_acc : 2.6287%
mths_since_last_delinq : 50.6261%
mths_since_recent_bc_dlq : 76.4419%
mths_since_recent_inq : 12.4832%
num_accts_ever_120_pd : 3.7512%
num_actv_bc_tl : 3.7512%
num_rev_accts : 3.7513%
num_sats : 3.1083%
num_tl_120dpd_2m : 7.5515%
num_tl_30dpd : 3.7512%
num_tl_90g_dpd_24m : 3.7512%
num_tl_op_past_12m : 3.7512%
open_il_24m : 47.2733%
open_rv_24m : 47.2733%
percent_bc_gt_75 : 3.6949%
pub_rec : 0.0000%
pub_rec_bankruptcies : 0.0389%
purpose : 0.0000%
revol_util : 0.0701%
tax_liens : 0.0023%
term : 0.0000%
title : 1.1501%
total_acc : 0.0000%
verification_status : 0.0000%
zip_code : 0.0001%
fico_mean : 0.00

In [9]:
for col in df1.columns:
    print(f'{col} : {df[col].isna().sum()/df.shape[0]*100:.4f}%')

acc_now_delinq : 0.0000%
addr_state : 0.0000%
annual_inc : 0.0000%
chargeoff_within_12_mths : 0.0031%
collections_12_mths_ex_med : 0.0031%
delinq_2yrs : 0.0000%
dti : 0.0000%
earliest_cr_line : 0.0000%
emp_length : 5.9796%
funded_amnt : 0.0000%
home_ownership : 0.0000%
inq_last_12m : 47.2734%
installment : 0.0000%
int_rate : 0.0000%
issue_d : 0.0000%
loan_amnt : 0.0000%
mort_acc : 2.6287%
mths_since_last_delinq : 50.6261%
mths_since_recent_bc_dlq : 76.4419%
mths_since_recent_inq : 12.4832%
num_accts_ever_120_pd : 3.7512%
num_actv_bc_tl : 3.7512%
num_rev_accts : 3.7513%
num_sats : 3.1083%
num_tl_120dpd_2m : 7.5515%
num_tl_30dpd : 3.7512%
num_tl_90g_dpd_24m : 3.7512%
num_tl_op_past_12m : 3.7512%
open_il_24m : 47.2733%
open_rv_24m : 47.2733%
percent_bc_gt_75 : 3.6949%
pub_rec : 0.0000%
pub_rec_bankruptcies : 0.0389%
purpose : 0.0000%
revol_util : 0.0701%
tax_liens : 0.0023%
term : 0.0000%
title : 1.1501%
total_acc : 0.0000%
verification_status : 0.0000%
zip_code : 0.0001%
fico_mean : 0.00

In [10]:
zero_data_set_sum=df.isnull().sum()
zero_data_set_percentage=df.isnull().sum()/df.shape[0]*100
zero_data_set=pd.DataFrame({'sum':zero_data_set_sum, 'percentage': zero_data_set_percentage})
zero=zero_data_set[zero_data_set['sum']>0].head(40)

In [11]:
imputer = SimpleImputer(strategy='constant', fill_value=-999)
column_to_impute=[]
for i in range(len(zero.index)):
    column_to_impute.append(zero.index[i])

In [12]:
for col in column_to_impute:
    imputed_column = imputer.fit_transform(df[[col]])
    df[col] = imputed_column

In [13]:
for col in column_to_impute:
    imputed_column = imputer.fit_transform(df1[[col]])
    df1[col] = imputed_column

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1199861 entries, 0 to 1199860
Data columns (total 44 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   acc_now_delinq              1199861 non-null  float64
 1   addr_state                  1199861 non-null  object 
 2   annual_inc                  1199861 non-null  float64
 3   chargeoff_within_12_mths    1199861 non-null  float64
 4   collections_12_mths_ex_med  1199861 non-null  float64
 5   delinq_2yrs                 1199861 non-null  float64
 6   dti                         1199861 non-null  float64
 7   earliest_cr_line            1199861 non-null  object 
 8   emp_length                  1199861 non-null  object 
 9   funded_amnt                 1199861 non-null  float64
 10  home_ownership              1199861 non-null  object 
 11  inq_last_12m                1199861 non-null  float64
 12  installment                 1199861 non-null  float64
 1

In [15]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 590977 entries, 0 to 590976
Data columns (total 44 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   acc_now_delinq              590977 non-null  float64
 1   addr_state                  590977 non-null  object 
 2   annual_inc                  590977 non-null  float64
 3   chargeoff_within_12_mths    590977 non-null  float64
 4   collections_12_mths_ex_med  590977 non-null  float64
 5   delinq_2yrs                 590977 non-null  float64
 6   dti                         590977 non-null  float64
 7   earliest_cr_line            590977 non-null  object 
 8   emp_length                  590977 non-null  object 
 9   funded_amnt                 590977 non-null  float64
 10  home_ownership              590977 non-null  object 
 11  inq_last_12m                590977 non-null  float64
 12  installment                 590977 non-null  float64
 13  int_rate      

In [16]:
x_train=df
x_test=df1

In [17]:
def get_categorical_indicies(x_train):
    cats = []
    for col in x_train.columns:
        if is_numeric_dtype(x_train[col]):
            pass
        else:
            cats.append(col)
    cat_indicies = []
    for col in cats:
        cat_indicies.append(x_train.columns.get_loc(col))
    return cat_indicies
categorical_indicies = get_categorical_indicies(x_train)

def convert_cats(x_train):
    cats = []
    for col in x_train.columns:
        if is_numeric_dtype(x_train[col]):
            pass
        else:
            cats.append(col)
    cat_indicies = []
    for col in cats:
        x_train[col] = x_train[col].astype('category')
convert_cats(x_train)

In [18]:
def get_categorical_indicies1(x_test):
    cats1 = []
    for col in x_test.columns:
        if is_numeric_dtype(x_test[col]):
            pass
        else:
            cats1.append(col)
    cat_indicies1 = []
    for col in cats1:
        cat_indicies1.append(x_test.columns.get_loc(col))
    return cat_indicies1
categorical_indicies1 = get_categorical_indicies1(x_test)

def convert_cats1(x_test):
    cats1 = []
    for col in x_test.columns:
        if is_numeric_dtype(x_test[col]):
            pass
        else:
            cats1.append(col)
    cat_indicies1 = []
    for col in cats1:
        x_test[col] = x_test[col].astype('category')
convert_cats1(x_test)

In [19]:
print(categorical_indicies)
print(categorical_indicies1)

[1, 7, 8, 10, 14, 33, 36, 37, 39, 40]
[1, 7, 8, 10, 14, 33, 36, 37, 39, 40]


In [20]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1199861 entries, 0 to 1199860
Data columns (total 44 columns):
 #   Column                      Non-Null Count    Dtype   
---  ------                      --------------    -----   
 0   acc_now_delinq              1199861 non-null  float64 
 1   addr_state                  1199861 non-null  category
 2   annual_inc                  1199861 non-null  float64 
 3   chargeoff_within_12_mths    1199861 non-null  float64 
 4   collections_12_mths_ex_med  1199861 non-null  float64 
 5   delinq_2yrs                 1199861 non-null  float64 
 6   dti                         1199861 non-null  float64 
 7   earliest_cr_line            1199861 non-null  category
 8   emp_length                  1199861 non-null  category
 9   funded_amnt                 1199861 non-null  float64 
 10  home_ownership              1199861 non-null  category
 11  inq_last_12m                1199861 non-null  float64 
 12  installment                 1199861 non-nu

In [21]:
x_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 590977 entries, 0 to 590976
Data columns (total 44 columns):
 #   Column                      Non-Null Count   Dtype   
---  ------                      --------------   -----   
 0   acc_now_delinq              590977 non-null  float64 
 1   addr_state                  590977 non-null  category
 2   annual_inc                  590977 non-null  float64 
 3   chargeoff_within_12_mths    590977 non-null  float64 
 4   collections_12_mths_ex_med  590977 non-null  float64 
 5   delinq_2yrs                 590977 non-null  float64 
 6   dti                         590977 non-null  float64 
 7   earliest_cr_line            590977 non-null  category
 8   emp_length                  590977 non-null  category
 9   funded_amnt                 590977 non-null  float64 
 10  home_ownership              590977 non-null  category
 11  inq_last_12m                590977 non-null  float64 
 12  installment                 590977 non-null  float64 
 13 

In [22]:
for col,name in enumerate(x_train):
    if col in categorical_indicies:
        pass
    else:
        x_train[name] = x_train[name].astype(str)

In [23]:
for col,name in enumerate(x_test):
    if col in categorical_indicies1:
        pass
    else:
        x_test[name] = x_test[name].astype(str)

In [24]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1199861 entries, 0 to 1199860
Data columns (total 44 columns):
 #   Column                      Non-Null Count    Dtype   
---  ------                      --------------    -----   
 0   acc_now_delinq              1199861 non-null  object  
 1   addr_state                  1199861 non-null  category
 2   annual_inc                  1199861 non-null  object  
 3   chargeoff_within_12_mths    1199861 non-null  object  
 4   collections_12_mths_ex_med  1199861 non-null  object  
 5   delinq_2yrs                 1199861 non-null  object  
 6   dti                         1199861 non-null  object  
 7   earliest_cr_line            1199861 non-null  category
 8   emp_length                  1199861 non-null  category
 9   funded_amnt                 1199861 non-null  object  
 10  home_ownership              1199861 non-null  category
 11  inq_last_12m                1199861 non-null  object  
 12  installment                 1199861 non-nu

In [25]:
x_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 590977 entries, 0 to 590976
Data columns (total 44 columns):
 #   Column                      Non-Null Count   Dtype   
---  ------                      --------------   -----   
 0   acc_now_delinq              590977 non-null  object  
 1   addr_state                  590977 non-null  category
 2   annual_inc                  590977 non-null  object  
 3   chargeoff_within_12_mths    590977 non-null  object  
 4   collections_12_mths_ex_med  590977 non-null  object  
 5   delinq_2yrs                 590977 non-null  object  
 6   dti                         590977 non-null  object  
 7   earliest_cr_line            590977 non-null  category
 8   emp_length                  590977 non-null  category
 9   funded_amnt                 590977 non-null  object  
 10  home_ownership              590977 non-null  category
 11  inq_last_12m                590977 non-null  object  
 12  installment                 590977 non-null  object  
 13 

In [26]:
train_dataset = cb.Pool(x_train,y_train,
                        cat_features=categorical_indicies)

In [27]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)

(1199861, 44)
(590977, 44)
(1199861,)


In [28]:
CAT=cb.CatBoostClassifier(metric_period=100,
                          task_type='GPU',
                          iterations=3700, 
                          learning_rate=0.02,
                          depth=10,
                         )
CAT.fit(train_dataset)

0:	learn: 0.6817071	total: 120ms	remaining: 7m 22s
100:	learn: 0.4458884	total: 11.1s	remaining: 6m 35s
200:	learn: 0.4382556	total: 22.1s	remaining: 6m 25s
300:	learn: 0.4354558	total: 33.1s	remaining: 6m 14s
400:	learn: 0.4333984	total: 44.1s	remaining: 6m 3s
500:	learn: 0.4316449	total: 55.2s	remaining: 5m 52s
600:	learn: 0.4301013	total: 1m 6s	remaining: 5m 41s
700:	learn: 0.4286599	total: 1m 17s	remaining: 5m 31s
800:	learn: 0.4272615	total: 1m 28s	remaining: 5m 19s
900:	learn: 0.4259601	total: 1m 39s	remaining: 5m 8s
1000:	learn: 0.4247651	total: 1m 50s	remaining: 4m 57s
1100:	learn: 0.4236239	total: 2m 1s	remaining: 4m 46s
1200:	learn: 0.4225338	total: 2m 12s	remaining: 4m 35s
1300:	learn: 0.4214322	total: 2m 23s	remaining: 4m 23s
1400:	learn: 0.4204150	total: 2m 33s	remaining: 4m 12s
1500:	learn: 0.4194122	total: 2m 44s	remaining: 4m 1s
1600:	learn: 0.4184382	total: 2m 56s	remaining: 3m 51s
1700:	learn: 0.4174547	total: 3m 7s	remaining: 3m 39s
1800:	learn: 0.4165201	total: 3m 1

In [29]:
predictions = CAT.predict(x_test)
predictions

array([1, 0, 0, ..., 0, 0, 0], dtype=int64)

In [30]:
predict_proba= CAT.predict_proba(x_test)
predict_proba

array([[0.43220884, 0.56779116],
       [0.72849381, 0.27150619],
       [0.84562546, 0.15437454],
       ...,
       [0.78807995, 0.21192005],
       [0.95032729, 0.04967271],
       [0.83404446, 0.16595554]])

In [31]:
answer=pd.DataFrame(predict_proba[:,1], columns=["loan_status"], dtype=None, copy=False)
answer.index.rename('index', inplace= True )
answer

,loan_status
index,
0,0.5678
1,0.2715
2,0.1544
3,0.0447
4,0.0698
...,...
590972,0.1041
590973,0.2400
590974,0.2119


In [32]:
# answer.to_csv('answer_ivan_pronin_sankt-peterburg.csv', float_format = '%.6f')

In [33]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.
